In [1]:
import numpy as np
from scipy import linalg
from astropy.io import fits

In [2]:
theory = np.genfromtxt('tests/sample_files/theory.txt')
binned = np.genfromtxt('tests/sample_files/bin_ordered_data.dat')
noisey = np.genfromtxt('tests/sample_files/noise_mean.txt')
covariance = np.genfromtxt('tests/sample_files/covariance.txt')
inv_covariance = np.genfromtxt('tests/sample_files/inv_covariance.txt')

In [3]:
print(np.allclose(binned, noisey, rtol = 0.0000001))

False


In [4]:
binned_re = binned.reshape(30,-1)
theory_re = theory.reshape(30,-1)

overall_order = ['xi_plus', 'xi_minus']
bin_order = ['bin_1_1', 
             'bin_2_1', 'bin_2_2', 
             'bin_3_1', 'bin_3_2', 'bin_3_3', 
             'bin_4_1', 'bin_4_2', 'bin_4_3', 'bin_4_4', 
             'bin_5_1', 'bin_5_2', 'bin_5_3', 'bin_5_4', 'bin_5_5']
ordered_bins = []
for order in overall_order:
    for bin in bin_order:
        ordered_bins.append(order + '_' + bin)

new_order = [None] * len(ordered_bins)

In [5]:
for i, proper in enumerate(theory_re):
    for j, binned in enumerate(binned_re):
        if np.allclose(proper, binned) is True:
            new_order[i] = ordered_bins[j]

In [6]:
covariance_transposed = np.transpose(covariance)
print(np.allclose(covariance_transposed, covariance, rtol = 0.0000001))

True


In [7]:
# Checking to see if the KCAP inverse covariance is symmetric

inv_covariance_transposed = np.transpose(inv_covariance)
print(np.allclose(inv_covariance, inv_covariance_transposed, rtol = 0.0000001))

False


In [8]:
# Checking to see if the numpy method of inverting a covariance produces the same inverse covariance as KCAP

inv_covariance_numpy = np.linalg.inv(covariance)
print(np.allclose(inv_covariance, inv_covariance_numpy, rtol = 0.0000001))

True


In [9]:
# Checking if the determinants are the same

det_cov = np.linalg.det(covariance)
det_inv_cov = np.linalg.det(inv_covariance)
print(np.allclose(det_cov, 1/det_inv_cov, rtol = 0.0000001))

True


/home/ruyi/anaconda3/envs/kcap_env/lib/python3.9/site-packages/numpy/linalg/linalg.py:2159: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [10]:
# Check if scipy inverse is the same as the KCAP inverse

inv_covariance_scipy = linalg.inv(covariance)
print(np.allclose(inv_covariance, inv_covariance_scipy, rtol = 0.0000001))
print(np.allclose(inv_covariance, inv_covariance_scipy, rtol = 0.00000001))

True
False


In [11]:
# Check if the scipy inverse is symmetric

scipy_inv_covariance_transposed = np.transpose(inv_covariance_scipy)
print(np.allclose(scipy_inv_covariance_transposed, inv_covariance_scipy, rtol = 0.0000001))

False


In [12]:
scipy_numpy_diff = inv_covariance_scipy - inv_covariance_numpy

In [13]:
boosted_covariance = np.copy(covariance)
for i, val in enumerate(boosted_covariance):
    # boosted_covariance[i][i] = boosted_covariance[i][i]*2*10**len(val)
    boosted_covariance[i][i] = boosted_covariance[i][i]*10**10

In [14]:
inv_boosted_covariance = linalg.inv(boosted_covariance)
print(np.allclose(np.transpose(inv_boosted_covariance), inv_boosted_covariance, rtol = 0.0000001))

True


In [15]:
unity = np.dot(inv_covariance, covariance)
symmetrised_inv_covariance = (inv_covariance + inv_covariance.T)/2
symmetrised_unity = np.dot(symmetrised_inv_covariance, covariance)
symmetrised_unity_det = np.linalg.det(symmetrised_unity)
unity_det = np.linalg.det(unity)

In [16]:
hdul = fits.open('tests/sample_files/kids_covariance.fits')
hdul.info()
cov_data = hdul[1].data

Filename: tests/sample_files/kids_covariance.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  COVMAT        1 ImageHDU        13   (270, 270)   float64   
  2  xiP           1 BinTableHDU     29   135R x 5C   [K, K, K, D, D]   
  3  xiM           1 BinTableHDU     29   135R x 5C   [K, K, K, D, D]   
  4  NZ_SOURCE     1 BinTableHDU     36   119R x 8C   [D, D, D, D, D, D, D, D]   


In [17]:
print(np.allclose(covariance, cov_data, rtol = 0.0000001))

True


In [18]:
w, v = np.linalg.eig(covariance)
inv_eigenvals = np.zeros(shape = (270,270))

for i, val in enumerate(w):
    if val > 1.*10**(-10):
        inv_eigenvals[i][i] += 1/val
    else:
        pass

inv_eigenvals_diag = np.diag(inv_eigenvals)

eigen_inv_covariance = np.dot(v, np.dot(inv_eigenvals, v.T))
print(np.allclose(eigen_inv_covariance, eigen_inv_covariance.T, 0.0000001))
eigen_unity = np.dot(eigen_inv_covariance, covariance)
eigen_unity_det = np.linalg.det(eigen_unity)
diag = np.diag(eigen_unity)
# inv_eigenvals = 1/w

True


In [19]:
L = np.linalg.cholesky(covariance)
identity = np.identity(270)
y = np.linalg.solve(L, identity)
x = np.linalg.solve(L.T, y)

print(np.allclose(x, x.T, 0.0000001))

True


In [20]:
cholesky_unity = np.dot(x, covariance)
cholestky_unity_det = np.linalg.det(cholesky_unity)

In [21]:
test_identity = np.identity(12)
np.linalg.det(test_identity)

1.0